# Day 7 - Lab 1: Advanced Agent Workflows with MCP (SOLUTION)

This solution notebook contains complete implementations for all challenges.

## Key Concepts Demonstrated:
1. **Resource-based context management**
2. **Idempotent tool operations**
3. **Deterministic agent behavior**

In [ ]:
# Complete environment setup
import sys
import os
import json
from datetime import datetime, timedelta
from dataclasses import dataclass, field, asdict
from typing import List, Dict, Optional, Any

project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# MCP imports with proper guards
try:
    from mcp.server import Server
    from mcp.server.fastmcp import FastMCP
    MCP_AVAILABLE = True
    print('✅ MCP imports successful')
except ImportError:
    print('⚠️ MCP not available - using mock implementation')
    MCP_AVAILABLE = False
    
    # Mock implementations for demonstration
    class FastMCP:
        def __init__(self, name):
            self.name = name
            self.resources = {}
            self.tools = {}
        
        def resource(self, path):
            def decorator(func):
                self.resources[path] = func
                return func
            return decorator
        
        def tool(self):
            def decorator(func):
                self.tools[func.__name__] = func
                return func
            return decorator

from utils import setup_llm_client, save_artifact
client, model_name, api_provider = setup_llm_client(model_name='gpt-4o')

In [ ]:
# Load all data
assets_path = 'assets'

with open(f'{assets_path}/onboarding_docs.json', 'r') as f:
    onboarding_docs = json.load(f)
with open(f'{assets_path}/roles_access_matrix.json', 'r') as f:
    roles_access = json.load(f)
with open(f'{assets_path}/training_catalog.json', 'r') as f:
    training_catalog = json.load(f)
with open(f'{assets_path}/new_hires_sample.json', 'r') as f:
    new_hires = json.load(f)

print(f'Loaded {len(new_hires["new_hires"])} new hires')

In [ ]:
# Complete schema definitions
@dataclass
class HireProfile:
    id: str
    name: str
    email: str
    role: str
    start_date: str
    manager: str
    location: str
    level: str = 'mid'
    department: str = ''
    
@dataclass
class AccessRequest:
    system: str
    hire_id: str
    role: str
    justification: str
    status: str = 'pending'
    created_at: str = field(default_factory=lambda: datetime.now().isoformat())
    request_id: str = field(default_factory=lambda: f'AR-{datetime.now().strftime("%Y%m%d%H%M%S")}')

@dataclass
class TrainingItem:
    course_id: str
    title: str
    duration: str
    due_date: str
    delivery: str = 'self-paced'
    status: str = 'not_started'

## SOLUTION: Challenge 1 - MCP Resources

In [ ]:
# Initialize MCP server with complete implementations
mcp_server = FastMCP('onboarding-server')

@mcp_server.resource('onboarding/docs')
async def get_onboarding_docs() -> Dict[str, Any]:
    """Return complete onboarding documentation."""
    return {
        'policies': onboarding_docs['policies'],
        'procedures': onboarding_docs['procedures'],
        'glossary': onboarding_docs['glossary'],
        'timestamp': datetime.now().isoformat()
    }

@mcp_server.resource('onboarding/roles')
async def get_roles_matrix() -> Dict[str, Any]:
    """Return role-based access requirements."""
    return {
        'roles': roles_access['roles'],
        'systems': roles_access['systems'],
        'timestamp': datetime.now().isoformat()
    }

@mcp_server.resource('onboarding/training')
async def get_training_catalog() -> Dict[str, Any]:
    """Return training catalog with courses."""
    return {
        'courses': training_catalog['courses'],
        'delivery_methods': training_catalog['delivery_methods'],
        'certification_tracks': training_catalog['certification_tracks'],
        'timestamp': datetime.now().isoformat()
    }

print('✅ All MCP resources implemented')

## SOLUTION: Challenge 2 - MCP Tools with Idempotency

In [ ]:
# Storage with idempotency support
access_requests_db = {}
training_schedules_db = {}

@mcp_server.tool()
async def create_access_request(system: str, hire_id: str, role: str) -> Dict[str, Any]:
    """Create access request with idempotency."""
    # Check for existing request (idempotency key)
    key = f'{hire_id}:{system}'
    
    if key in access_requests_db:
        return {
            'status': 'already_exists',
            'request': asdict(access_requests_db[key]),
            'message': 'Request already exists (idempotent)'
        }
    
    # Create new request
    request = AccessRequest(
        system=system,
        hire_id=hire_id,
        role=role,
        justification=f'New hire onboarding for {role} role'
    )
    
    access_requests_db[key] = request
    
    return {
        'status': 'created',
        'request': asdict(request),
        'message': 'Access request created successfully'
    }

@mcp_server.tool()
async def schedule_training(hire_id: str, course_ids: List[str]) -> Dict[str, Any]:
    """Schedule training with validation."""
    scheduled = []
    errors = []
    
    # Get hire start date
    hire = next((h for h in new_hires['new_hires'] if h['id'] == hire_id), None)
    if not hire:
        return {'status': 'error', 'message': f'Hire {hire_id} not found'}
    
    start_date = datetime.fromisoformat(hire['start_date'])
    
    for course_id in course_ids:
        # Find course
        course = next((c for c in training_catalog['courses'] if c['id'] == course_id), None)
        if not course:
            errors.append(f'Course {course_id} not found')
            continue
        
        # Check if already scheduled (idempotency)
        key = f'{hire_id}:{course_id}'
        if key in training_schedules_db:
            scheduled.append(training_schedules_db[key])
            continue
        
        # Create training item
        due_date = start_date + timedelta(days=course.get('due_within_days', 30))
        item = TrainingItem(
            course_id=course_id,
            title=course['title'],
            duration=course['duration'],
            due_date=due_date.isoformat(),
            delivery=course.get('delivery', 'self-paced')
        )
        
        training_schedules_db[key] = item
        scheduled.append(item)
    
    return {
        'status': 'success',
        'scheduled': [asdict(item) for item in scheduled],
        'errors': errors
    }

print('✅ MCP tools with idempotency implemented')

## SOLUTION: Challenge 3 - Reliability Demonstration

In [ ]:
import random

def check_soc2_requirement_unstructured(role: str, context: str) -> bool:
    """Unreliable: Inconsistent interpretation of ambiguous context."""
    # Ambiguous words lead to random decisions
    if any(word in context.lower() for word in ['maybe', 'might', 'possibly', 'could']):
        return random.choice([True, False])
    
    # Otherwise try to parse context
    if 'soc2' in context.lower() or 'compliance' in context.lower():
        return True
    return role in ['engineer', 'security']

def check_soc2_requirement_mcp(role: str) -> bool:
    """Reliable: Always checks authoritative MCP resource."""
    # Always consult the structured resource
    for policy in onboarding_docs['policies']:
        if policy['id'] == 'POL-003':  # SOC2 policy
            return role in policy['required_for_roles']
    return False

# Demonstrate consistency difference
print('Testing consistency with 10 runs each:\n')

test_cases = [
    ('engineer', 'The new engineer might need SOC2 training, maybe check with compliance'),
    ('data-analyst', 'Data analyst possibly requires compliance training'),
    ('marketing', 'Marketing could need SOC2 but unsure')
]

for role, context in test_cases:
    unstructured_results = [check_soc2_requirement_unstructured(role, context) for _ in range(10)]
    mcp_results = [check_soc2_requirement_mcp(role) for _ in range(10)]
    
    print(f'Role: {role}')
    print(f'  Unstructured (varied): {set(unstructured_results)} - Consistent: {len(set(unstructured_results)) == 1}')
    print(f'  MCP (deterministic): {set(mcp_results)} - Consistent: {len(set(mcp_results)) == 1}')
    print()

# Assert MCP is always consistent
assert all(len(set([check_soc2_requirement_mcp(r) for _ in range(100)])) == 1 
           for r in ['engineer', 'marketing', 'data-analyst']), \
    'MCP should always be deterministic'

print('✅ MCP determinism verified')

## Integration Test: Complete Onboarding Workflow

In [ ]:
async def onboard_new_hire(hire_data: Dict) -> Dict:
    """Complete onboarding workflow using MCP."""
    hire_id = hire_data['id']
    role = hire_data['role']
    
    plan = {
        'hire_id': hire_id,
        'hire_name': hire_data['name'],
        'access_requests': [],
        'training_schedule': [],
        'required_policies': []
    }
    
    # 1. Get role requirements from MCP resource
    role_data = roles_access['roles'].get(role, {})
    required_systems = role_data.get('required_systems', [])
    
    # 2. Create access requests using MCP tool
    for system in required_systems:
        result = await create_access_request(system, hire_id, role)
        plan['access_requests'].append(result)
    
    # 3. Get required training from MCP resource
    required_courses = []
    for course in training_catalog['courses']:
        if 'all' in course.get('required_for_roles', []) or role in course.get('required_for_roles', []):
            required_courses.append(course['id'])
    
    # 4. Schedule training using MCP tool
    if required_courses:
        result = await schedule_training(hire_id, required_courses)
        plan['training_schedule'] = result.get('scheduled', [])
    
    # 5. Get required policies from MCP resource
    for policy in onboarding_docs['policies']:
        if 'all' in policy.get('required_for_roles', []) or role in policy.get('required_for_roles', []):
            plan['required_policies'].append({
                'id': policy['id'],
                'title': policy['title'],
                'url': policy['url']
            })
    
    return plan

# Test with first hire
import asyncio
test_hire = new_hires['new_hires'][0]
onboarding_plan = await onboard_new_hire(test_hire)

print(f"Onboarding Plan for {onboarding_plan['hire_name']}:")
print(f"  - {len(onboarding_plan['access_requests'])} access requests created")
print(f"  - {len(onboarding_plan['training_schedule'])} training courses scheduled")
print(f"  - {len(onboarding_plan['required_policies'])} policies to review")

# Test idempotency - run again
plan2 = await onboard_new_hire(test_hire)
idempotent = all(r['status'] == 'already_exists' 
                 for r in plan2['access_requests'] 
                 if 'status' in r)
print(f"\n✅ Idempotency verified: {idempotent}")

## Key Insights

### Why MCP Improves Reliability:

1. **Structured Resources**: Data is always accessed the same way
2. **Validated Tools**: Operations have consistent signatures  
3. **Idempotency**: Same operation can be safely retried
4. **Schema Enforcement**: Data structures are predictable

### Production Considerations:

- **Versioning**: Resources should include version info
- **Caching**: Resources can be cached for performance
- **Audit Trail**: All tool calls should be logged
- **Error Handling**: Tools should return structured errors
- **Rate Limiting**: Protect resources from overuse